## Import packages

In [1]:
import selenium
from selenium import webdriver

import shutil, time, os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from importlib import reload

import src
reload(src)

<module 'src' from 'C:\\Users\\Matth\\git\\DataAnalysisWorkbooks\\warcraftLogs\\src.py'>

## Configurations

In [2]:
# Set path variables
path_to_adblocker = r"C:\Users\Matth\Desktop\3.12_0"
path_to_download_dir = r"C:\Users\Matth\Downloads"

In [9]:
N_parses = 11

## Navigate to WCL

In [5]:
boss = "High Warlord Naj'entus"

In [6]:
boss_link_dict = {"High Warlord Naj'entus" : "#boss=601", "Supremus" : "#boss=602", "Shade of Akama" : "#boss=603", 
                  "Teron Gorefiend" : "#boss=604", "Gurtogg Bloodboil" : "#boss=605", "Reliquary of Souls" : "#boss=606", 
                  "Mother Shahraz" : "#boss=607", "The Illidari Council" : "#boss=608", "Illidan Stormrage" : "#boss=609", 
                  "Rage Winterchill" : "#boss=618", "Anetheron" : "#boss=619", "Kaz'rogal" : "#boss=620", 
                  "Azgalor" : "#boss=621", "Archimonde" : "#boss=622"}

In [7]:
boss_page_url, browser = src.load_top_N_scraper(path_to_adblocker, boss, boss_link_dict)

## Main code loop

In [ ]:
def main():
    
    for i in range(1, N_parses):
        rank, char_info, date, HPS, duration = src.get_boss_data_top_N_scraper(browser, i)

        name = char_info.split("\n")[0]

        server_region = char_info.split("-")

        if len(server_region) == 1:
            server_region = char_info.split("\n")[1]
            server = server_region.split(" ")[0]
            region = server_region.split(" ")[1]

        else:
            server_region = char_info.split("-")[1]
            server = server_region.split(" ")[1]
            region = server_region.split(" ")[2]
        
        print(rank, name, server, region)
        
        if src.check_if_parse_already_recorded_top_N(boss, rank, name): 
                print("Parse already recorded. Skipping")
                continue
    
        link = browser.find_element_by_link_text(name)
        link.click()
        time.sleep(1)
    
        player_df = pd.DataFrame(pd.np.empty((0, 16)))
        player_df.columns = ["Rank", "Name", "Server", "Date", "Kill time", "nHealers", "Spriest?", "Innervate?", "LB_uptime", "HPS", "% LB (tick) HPS", "% LB (bloom) HPS", "% Rejuv HPS", "% Regrowth HPS", "% Swiftmend HPS", "Rotations"]
    
        temp_url = browser.current_url
        
        boss_tanks = src.get_tanks(browser)
        nHealers = src.get_nHealers(browser)
    
        print(f"Tanks: {boss_tanks}")
        
        browser.get(temp_url)
        time.sleep(0.5)
        
        player_link = browser.find_element_by_link_text(name)
        player_link.click()
        time.sleep(0.5)
        
        # Scrape spell HPS
        LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, LB_uptime = src.get_spell_info(browser, HPS)
        time.sleep(2)
                   
        # Scrape mana sources
        spriest = src.check_spriest(browser)
        innervate = src.check_innervate(browser)
        time.sleep(0.5)    
        
        # Download the cast-sequence CSV.
        src.download_csv(browser, temp_url, "filter-casts-tab", path_to_download_dir, "character_data/cast_sequence.csv")
        time.sleep(1)
        
        # Clean the csv
        df = src.clean_cast_sequence_csv()
        df = src.fix_cast_time(df)
        time.sleep(0.5)
        
        # Get the rotations
        rotations = src.calculate_rotations(df, boss, boss_tanks)
        print(f'Rotations: {rotations}')  

        to_append = [rank, name, server + " " + region, date, duration, str(nHealers), spriest, innervate, LB_uptime, HPS, LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, rotations]
        src.export_to_excel(boss, to_append, player_df, name, 'top_N_druids')
        
        os.remove(f"character_data/{boss.replace(' ', '')}_{name}.csv")
        os.remove(f"character_data/cast_sequence.csv")
        
        print('-------')
        browser.get(boss_page_url)
        time.sleep(1)
        
        
    print(f"{boss} scraping complete.")
    print("---------------")
    
    
if __name__ == "__main__":
    main()

1 一杯马爹利 雷霆之击 (CN)
Parse already recorded. Skipping
2 氢硫化钠 奥罗 (CN)
Parse already recorded. Skipping
3 搞不得 末日之刃 (CN)
Parse already recorded. Skipping
4 喵绿喵绿喵 范沃森 (CN)
Parse already recorded. Skipping
5 Дарнейм Пламегор (EU)
Parse already recorded. Skipping
6 芬巴巴之花 碧空之歌 (CN)
Parse already recorded. Skipping
7 黑夜现苍鹰 伦鲁迪洛尔 (CN)
Parse already recorded. Skipping
8 德国进口德 维希度斯 (CN)
Parse already recorded. Skipping
9 托尔德 雷霆之击 (CN)
Parse already recorded. Skipping
10 Day 艾隆纳亚 (CN)


In [11]:
browser.get(boss_page_url)